<a href="https://colab.research.google.com/github/ld-minh4354/Impact-of-AI-on-Environmental-Quality/blob/main/Impact_of_artificial_intelligence_on_environmental_quality_A_mathematical_modeling_approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set-ups

In [ ]:
# Importing relevant Python packages
import math
import sympy as sym
from scipy.optimize import fsolve, bisect
import matplotlib.pyplot as plt

In [ ]:
# Simulation constants
period_length = 5

# Economy constants
sigma = 10
alpha = 1/3
psi = alpha ** 2
phi = (1 - alpha) * (1 - sigma)
Y_c0 = 307.77
Y_d0 = 1893.25

# Calculating A_c0 and A_d0
r = (Y_c0 / Y_d0) ** (1 / (alpha + phi -1))
A_c0 = (alpha ** 2 / psi) ** (alpha / (1-alpha)) * (1 + r ** (-phi)) ** (-(alpha + phi) / phi) * Y_c0
A_d0 = r * A_c0

# Innovation constants
gamma = 0.04
eta_c = eta_d = 0.5 * period_length

# Environment constants
temp_disaster = 6
emission_0 = 17.48251782
S_bar = 280 * (2 ** (temp_disaster / 3) - 1)
S_0 = S_bar - 99
delta = 0.5 * emission_0 / S_0
xi = emission_0 / Y_d0
CO2_disaster = 1120

# Simulation Code - Helper Functions

In [ ]:
# This calculates the ratio of expected profit for researchers in the new and old sectors. See Lemma 8.
def scientist_allocation_equilibrium(s_c, A_c, A_d, t, k):
    I_ct = math.e ** (3 * k * t)
    I_dt = math.e ** (k * t)

    func = (eta_c / eta_d) \
         * (1 + gamma * I_ct) / (1 + gamma * I_dt) \
         * ((1 + gamma * eta_c * I_ct * s_c) / (1 + gamma * eta_d * I_dt * (1 - s_c))) ** (-phi - 1) \
         * (A_c / A_d) ** (-phi)

    return func

In [ ]:
# This simulates the equilibrium process and determine the values of variables at equilibrium
def simulation_equilibrium(k, num_period, test = False):
    # Initialise arrays of different variables, some of which have starting values at t=0
    A_c = [A_c0]
    A_d = [A_d0]
    s_c = []
    Y_ratio = []
    Y_d = []
    C = []
    S = [S_0]
    temp = []


    for t in range(1, num_period + 1):
        # Solving for equilibrium scientist allocation
        if scientist_allocation_equilibrium(1, A_c[t-1], A_d[t-1], t, k) >= 1:
            s_ct = 1
        elif scientist_allocation_equilibrium(0, A_c[t-1], A_d[t-1], t, k) <= 1:
            s_ct = 0
        else:
            equation = lambda s_ct: scientist_allocation_equilibrium(s_ct, A_c[t-1], A_d[t-1], t, k) - 1
            s_ct = bisect(equation, 0, 1)

        # Determining the value of I_jt
        I_ct = math.e ** (3 * k * t)
        I_dt = math.e ** (k * t)

        # Calculating values of relevant variables at time t
        A_ct = (1 + gamma * eta_c * I_ct * s_ct) * A_c[-1]
        A_dt = (1 + gamma * eta_d * I_dt * (1 - s_ct)) * A_d[-1]

        Y_ct = (alpha ** 2 / psi) ** (alpha / (1-alpha)) * (1 / (A_ct ** phi + A_dt ** phi) ** ((alpha + phi) / phi)) * A_ct * A_dt ** (alpha + phi)
        Y_dt = (alpha ** 2 / psi) ** (alpha / (1-alpha)) * (1 / (A_ct ** phi + A_dt ** phi) ** ((alpha + phi) / phi)) * A_ct ** (alpha + phi) * A_dt
        Y_ratio_t = Y_ct / (Y_ct + max(Y_dt, 0.0000001))

        C_t = (alpha ** 2 / psi) ** (alpha / (1-alpha)) * (1 - alpha ** 2) * (A_ct ** phi + A_dt ** phi) ** (-1 / phi) * A_ct * A_dt

        S_t = min(max(-xi * Y_dt + (1 + delta) * S[-1], 0.00001), S_bar)
        temp_t = 3 * math.log2((CO2_disaster - S_t) / 280)

        # Append these values to their arrays
        s_c.append(s_ct)
        A_c.append(A_ct)
        A_d.append(A_dt)
        Y_d.append(Y_dt)
        Y_ratio.append(Y_ratio_t)
        C.append(C_t)
        S.append(S_t)
        temp.append(temp_t)

    if test:
        return C[-1]

    # Return dictionary containing the arrays, with elements at t=0 removed
    return {'A_c': A_c[1:], 'A_d': A_d[1:], 's_c': s_c, 'Y_d': Y_d, 'Y_ratio': Y_ratio, 'C': C, 'S': S, 'temp': temp}

In [ ]:
# Function to plot graph of each variable over time, with the cases of no and with AI plotted together
# Optinal to specify whether to use log scale for y-axis (default is No) and the year to plot until (default is plotting all years)
def plot_equilibrium(variable, label, log_scale = False):
    plt.plot(equilibrium[0][variable], label = 'Acemoglu (\'24)')
    plt.plot(equilibrium[1][variable], label = 'Goldman Sachs (\'23)')
    plt.plot(equilibrium[2][variable], label = 'McKinsey (\'23)')
    plt.plot(equilibrium[3][variable], label = 'Korinek & Suh (\'24)')

    if log_scale:
        plt.yscale('log')
    else:
        plt.yscale('linear')

    plt.xlabel('Year')
    plt.ylabel(label)

    plt.legend()
    plt.show()

# Simulation and Graph Plotting

In [ ]:
C_original = simulation_equilibrium(0, 4, test = True)

def find_k(gdp_effect):
    equation = lambda k: simulation_equilibrium(k, 4, test = True) - (1 + gdp_effect / 100) * C_original
    k = fsolve(equation, x0 = gdp_effect)
    return k

In [ ]:
gdp_effect_list = [0.9, 7, 33.2, 64.4]
equilibrium = []

def main():
    for gdp_effect in gdp_effect_list:
        k = find_k(gdp_effect)
        print(k)
        equilibrium.append(simulation_equilibrium(k, 20))

main()

<ipython-input-4-6e6ccc1669d0>:39: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  temp_t = 3 * math.log2((CO2_disaster - S_t) / 280)
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_minpack_py.py:177: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)
<ipython-input-3-874fee8a85be>:9: RuntimeWarning: overflow encountered in power
  * (A_c / A_d) ** (-phi)
<ipython-input-3-874fee8a85be>:6: RuntimeWarning: overflow encountered in multiply
  func = (eta_c / eta_d) \


[0.2056726]
[0.06876494]


<ipython-input-3-874fee8a85be>:8: RuntimeWarning: overflow encountered in power
  * ((1 + gamma * eta_c * I_ct * s_c) / (1 + gamma * eta_d * I_dt * (1 - s_c))) ** (-phi - 1) \
<ipython-input-4-6e6ccc1669d0>:29: RuntimeWarning: overflow encountered in multiply
  A_ct = (1 + gamma * eta_c * I_ct * s_ct) * A_c[-1]
<ipython-input-4-6e6ccc1669d0>:34: RuntimeWarning: invalid value encountered in divide
  Y_ratio_t = Y_ct / (Y_ct + max(Y_dt, 0.0000001))


ValueError: The function value at x=0.0 is NaN; solver cannot continue.

In [ ]:
plot_equilibrium('C', 'Total consumption', log_scale = True)

In [ ]:
# Plotting variables
#plot_equilibrium('s_c', 'Proportion of scientists in the new sector')
#plot_equilibrium('Y_d', 'Production of old input')
#plot_equilibrium('Y_ratio', 'Proportion of new input production in total production', end_year = 150)
#plot_equilibrium('C', 'Total consumption', log_scale = True, end_year = 150)
#plot_equilibrium('S', 'Quality of the environment')
#plot_equilibrium('temp', 'Increase in temperature')